# Get Data

In [ ]:
import requests
import pandas as pd
import geopandas as gpd

In [ ]:
# Data Sources

# CMS - General Hospital Info
# https://data.cms.gov/provider-data/dataset/xubh-q36u
hosp_csv_url = 'https://data.cms.gov/provider-data/sites/default/files/resources/893c372430d9d71a1c52737d01239d47_1729022728/Hospital_General_Information.csv'

# CMS - Maternal Health
# https://data.cms.gov/provider-data/dataset/nrdb-3fcy
mat_csv_url = "https://data.cms.gov/provider-data/sites/default/files/resources/5a4754b088fdb10d2ae278ef215925a7_1729022763/Maternal_Health-Hospital.csv"

# ArcGIS Hospital Data API
# https://hifld-geoplatform.hub.arcgis.com/
hifld_url = "https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Hospitals_gdb/FeatureServer/0/query"

# Shapefile Paths
# downloaded from https://www.weather.gov/gis/AWIPSShapefiles
state_shapefile_path = 'shapefiles/s_18mr25/s_18mr25.shp'
county_shapefile_path = 'shapefiles/c_18mr25/c_18mr25.shp'

In [ ]:
def download_csv_to_pandas(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Load the CSV data if the request was successful
        df = pd.read_csv(url)
        print(f"Data from {url} loaded successfully", df.shape)
        return df
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred while fetching {url}: {http_err}")
    except Exception as err:
        print(f"An error occurred while fetching {url}: {err}")
    return None


def download_arcgis_api_to_pandas(url):
    params = {
        "where": "1=1",       
        "outFields": "*",    
        "f": "geojson"        
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an error for bad status codes

        # Load GeoJSON data into a GeoDataFrame
        gdf = gpd.read_file(response.text)
        print(f"Data from {url} loaded successfully", gdf.shape)
        return gdf
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    return None

cms_mat_df = download_csv_to_pandas(mat_csv_url)
cms_hosp_df = download_csv_to_pandas(hosp_csv_url)
hifld_df = download_arcgis_api_to_pandas(hifld_url)

# Create US & Maryland Maps

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
# US Heatmap
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles='Cartodb Positron')

df = hifld_df
df['lon'] = df['geometry'].apply(lambda point: point.x)
df['lat'] = df['geometry'].apply(lambda point: point.y)
heat_data = df[['lat', 'lon']].values.tolist()
HeatMap(heat_data, radius=2, blur=1).add_to(m)

m.save('outputs/us_heatmap.html')

In [ ]:
# Maryland Heatmap
m = folium.Map(location=[39.0458, -76.6413], zoom_start=8, tiles='Cartodb Positron')

# Hospital data
df = hifld_df
md_df = df[df['STATE'] == 'MD'].copy()
md_df['lon'] = md_df['geometry'].apply(lambda point: point.x)
md_df['lat'] = md_df['geometry'].apply(lambda point: point.y)

heat_data = md_df[['lat', 'lon']].values.tolist()
HeatMap(heat_data, radius=8, blur=4).add_to(m)

# Add Maryland boundary to the map
states_gdf = gpd.read_file(state_shapefile_path)
md_boundary = states_gdf[states_gdf['NAME'] == 'Maryland']

folium.GeoJson(
    md_boundary,
    style_function=lambda feature: {
        'fillColor': 'none',  
        'color': 'blue',   
        'weight': 1        
    }
).add_to(m)

m.save('outputs/maryland_heatmap.html')

In [ ]:
# Maryland Cloropath
m = folium.Map(location=[39.0458, -76.6413], zoom_start=8, tiles='Cartodb Positron')

# Load the shapefile for US counties
counties_gdf = gpd.read_file(county_shapefile_path)
md_counties = counties_gdf[counties_gdf['STATE'] == 'MD']

# Filter hospital data for Maryland
md_df = df[df['STATE'] == 'MD'].copy()

# Spatial join to assign counties to hospitals
md_counties = md_counties.to_crs(epsg=4326)
md_df = gpd.sjoin(md_df, md_counties, how="left", predicate='within')

# Aggregate count of hospitals by County
hospital_count_by_county = md_df.groupby('COUNTYNAME').size().reset_index(name='HOSPITAL_COUNT')

# Merge aggregated hospital count data with county shapefile
md_counties = md_counties.merge(hospital_count_by_county, on='COUNTYNAME')

# Create Choropleth map based on hospital count by county
folium.Choropleth(
    geo_data=md_counties,
    data=md_counties,
    columns=['COUNTYNAME', 'HOSPITAL_COUNT'],
    key_on='feature.properties.COUNTYNAME',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='Number of Hospitals by County',
    highlight=True,
    bins=[1, 3, 5, 10, 15, 20],
).add_to(m)

# Save or display the map
m.save('outputs/maryland_choropleth.html')

In [ ]:
# Maryland Dot Map
m = folium.Map(location=[39.0458, -76.6413], zoom_start=8, tiles='Cartodb Positron')

# Hospital data
df = hifld_df
md_df = df[df['STATE'] == 'MD'].copy()
md_df['lon'] = md_df['geometry'].apply(lambda point: point.x)
md_df['lat'] = md_df['geometry'].apply(lambda point: point.y)

for idx, row in md_df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=4,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=row.get('NAME', 'Hospital')
    ).add_to(m)

# Add Maryland boundary to the map
states_gdf = gpd.read_file(state_shapefile_path)
md_boundary = states_gdf[states_gdf['NAME'] == 'Maryland']

folium.GeoJson(
    md_boundary,
    style_function=lambda feature: {
        'fillColor': 'none',  
        'color': 'blue',   
        'weight': 1        
    }
).add_to(m)

m.save('outputs/maryland_marker.html')

# Combine Datasets

In [ ]:
from fuzzywuzzy import process, fuzz
import numpy as np

In [ ]:
# Clean Data
hifld_df_md = hifld_df[hifld_df['STATE'] == 'MD']
cms_hosp_df_md = cms_hosp_df[cms_hosp_df['State'] == 'MD']
cms_mat_df_md = cms_mat_df[cms_mat_df['State'] == 'MD']

cms_hosp_df_md = cms_hosp_df_md[['Facility ID', 'Facility Name', 'Address', 'City/Town', 'State', 'ZIP Code', 'County/Parish',
'Hospital Type','Hospital Ownership', 'Emergency Services','Meets criteria for birthing friendly designation','Hospital overall rating']]

cms_mat_df_md['Score'] = pd.to_numeric(cms_mat_df_md['Score'], errors='coerce')
cms_mat_df_md = cms_mat_df_md.pivot_table(index=['Facility ID', 'Facility Name', 'Address', 'City/Town', 'State', 'ZIP Code'],
                         columns='Measure Name',
                         values='Score').reset_index()
cms_mat_df_md.columns.name = None
cms_mat_df_md.columns = [str(col) for col in cms_mat_df_md.columns]

In [ ]:
df1 = hifld_df_md 
df2 = cms_hosp_df_md
df3 = cms_mat_df_md

df1.rename(columns={
    'NAME': 'Facility Name',
    'ADDRESS': 'Address',
    'CITY': 'City/Town',
    'STATE': 'State',
    'ZIP': 'ZIP Code'
}, inplace=True)

df1['ZIP Code'] = df1['ZIP Code'].astype(str)
df2['ZIP Code'] = df2['ZIP Code'].astype(str)
df3['ZIP Code'] = df3['ZIP Code'].astype(str)

# Combine CMS datasets
df2_df3_merged = pd.merge(df2, df3, on=['Address', 'City/Town', 'ZIP Code'], how='outer', suffixes=('_df2', '_df3'))
for col in df2.columns:
    if col in df3.columns and col not in ['Address', 'City/Town', 'ZIP Code']:
        col_df2 = f"{col}_df2"
        col_df3 = f"{col}_df3"
        
        if col_df2 in df2_df3_merged.columns and col_df3 in df2_df3_merged.columns:
            # Where values are equal or one is NaN, fill NaN or keep one column
            df2_df3_merged[col] = df2_df3_merged[col_df2].combine_first(df2_df3_merged[col_df3])
            
            # Drop the old columns with suffixes
            df2_df3_merged.drop(columns=[col_df2, col_df3], inplace=True)

# Fuzzy Matching Addresses Between df1 and df2_df3_merged
def get_best_match(address, choices, threshold=85):
    """Find the best fuzzy match for an address."""
    match, score = process.extractOne(address, choices, scorer=fuzz.token_sort_ratio)
    return match if score >= threshold else None

address_choices = df2_df3_merged['Address'].dropna().unique()
df1['Fuzzy_Matched_Address'] = df1['Address'].apply(lambda x: get_best_match(x, address_choices))

# Combine all
final_merge = pd.merge(df1, df2_df3_merged, left_on=['Fuzzy_Matched_Address', 'ZIP Code'],
                       right_on=['Address', 'ZIP Code'], how='outer', suffixes=('_cms', '_hifld'))

print(df1.count().max())
print(df2.count().max())
print(df3.count().max())
print(final_merge.count().max())

In [ ]:
# Helper function to consolidate primary values, ignoring 'Not Available'
def consolidate(primary, alt1, alt2=None):
    values = [primary, alt1, alt2]
    # Filter out NaN and 'Not Available'
    values = [v for v in values if pd.notna(v) and v != 'Not Available']
    return values[0] if values else np.nan  # Return first valid value or NaN if none

# Helper function to get alternatives, ignoring 'Not Available'
def get_alternative(primary, alt1, alt2=None):
    alternatives = []
    for alt in [alt1, alt2]:
        if pd.notna(alt) and alt != 'Not Available' and alt != primary:
            alternatives.append(alt)
        else:
            alternatives.append(np.nan)
    return alternatives[0]  # Return the first valid alternative, or NaN if none

# Consolidate Facility Name
final_merge['Facility Name'] = final_merge.apply(
    lambda row: consolidate(row['Facility Name_cms'], row['Facility Name_hifld'], row['ALT_NAME']), axis=1)

final_merge['Facility Name_alt'] = final_merge.apply(
    lambda row: get_alternative(row['Facility Name'], row['Facility Name_hifld'], row['ALT_NAME']), axis=1)

# Consolidate Address
final_merge['Address'] = final_merge.apply(
    lambda row: consolidate(row['Address_cms'], row['Address_hifld'], row['Fuzzy_Matched_Address']), axis=1)

final_merge['Address_alt'] = final_merge.apply(
    lambda row: get_alternative(row['Address'], row['Address_hifld'], row['Fuzzy_Matched_Address']), axis=1)

# Consolidate City/Town
final_merge['City/Town'] = final_merge.apply(
    lambda row: consolidate(row['City/Town_cms'], row['City/Town_hifld']), axis=1)

final_merge['City/Town_alt'] = final_merge.apply(
    lambda row: get_alternative(row['City/Town'], row['City/Town_hifld']), axis=1)

# Consolidate State
final_merge['State'] = final_merge.apply(
    lambda row: consolidate(row['State_cms'], row['State_hifld']), axis=1)

final_merge['State_alt'] = final_merge.apply(
    lambda row: get_alternative(row['State'], row['State_hifld']), axis=1)

# Consolidate County
final_merge['County'] = final_merge.apply(
    lambda row: consolidate(row['County/Parish'], row['COUNTY']), axis=1)

final_merge['County_alt'] = final_merge.apply(
    lambda row: get_alternative(row['County'], row['COUNTY']), axis=1)

# Keep ZIP Code and ZIP4 as is
final_merge['ZIP Code'] = final_merge['ZIP Code'].fillna(final_merge['ZIP4'])

final_merge.drop(columns=[col for col in final_merge.columns if col.endswith('_cms') or col.endswith('_hifld')], inplace=True)

In [ ]:
final_merge = final_merge[['OBJECTID', 'ID', 'Facility ID', 'Facility Name', 'Facility Name_alt',
       'Address', 'Address_alt', 'City/Town', 'City/Town_alt', 'State',
       'State_alt', 'County', 'County_alt', 'ZIP Code', 'ZIP4', 'TYPE',
       'OWNER', 'Hospital Type', 'Hospital Ownership', 'STATUS', 'POPULATION',
       'NAICS_CODE', 'NAICS_DESC', 'WEBSITE', 'TELEPHONE', 'TTL_STAFF', 'BEDS',
       'TRAUMA', 'HELIPAD', 'Emergency Services',
       'Meets criteria for birthing friendly designation',
       'Hospital overall rating', 'Cesarean Birth', 'Elective Delivery',
       'Exclusive Breast Milk Feeding',
       'Risk Adjusted Severe Obstetric Complications (All)',
       'Risk Adjusted Severe Obstetric Complications (excluding blood-transfusion-only cases)',
       'LATITUDE', 'LONGITUDE', 'geometry']]

# Clean up Coordinates

In [ ]:
from geopy.geocoders import Nominatim
import time

In [ ]:
# Clean Up Coordinates
df = final_merge
def extract_coords(row):
    if pd.isna(row['LATITUDE']) or pd.isna(row['LONGITUDE']):
        if pd.notna(row['geometry']):
            # Extract longitude and latitude from 'geometry' string (format: 'POINT (lon lat)')
            lon, lat = row['geometry'].replace('POINT (', '').replace(')', '').split()
            return pd.Series({'LATITUDE': float(lat), 'LONGITUDE': float(lon)})
    return pd.Series({'LATITUDE': row['LATITUDE'], 'LONGITUDE': row['LONGITUDE']})

# Apply extraction to fill missing LAT/LON
df[['LATITUDE', 'LONGITUDE']] = df.apply(extract_coords, axis=1)

In [ ]:
# Initialize geocoder with a user agent
geolocator = Nominatim(user_agent="hospital_locator")

# Function to build the full address string from available fields
def build_full_address(row):
    parts = [
        row.get('Address') or row.get('Address_alt'), 
        row.get('City/Town') or row.get('City/Town_alt'),
        row.get('County') or row.get('County_alt'),
        row.get('State') or row.get('State_alt'),
        row.get('ZIP Code') or row.get('ZIP4')
    ]
    # Filter out None or NaN values and join into a single string
    return ', '.join([str(part) for part in parts if pd.notna(part)])

# Function to geocode missing LATITUDE and LONGITUDE
def geocode_missing(row):
    if pd.isna(row['LATITUDE']) or pd.isna(row['LONGITUDE']):
        full_address = build_full_address(row)
        try:
            location = geolocator.geocode(full_address, timeout=10)
            if location:
                return pd.Series({'LATITUDE': location.latitude, 'LONGITUDE': location.longitude})
        except Exception as e:
            print(f"Error geocoding address: {full_address}, Error: {e}")
        # Return NaN if geocoding fails
        return pd.Series({'LATITUDE': np.nan, 'LONGITUDE': np.nan})
    return pd.Series({'LATITUDE': row['LATITUDE'], 'LONGITUDE': row['LONGITUDE']})

# Apply geocoding to rows with missing coordinates
df[['LATITUDE', 'LONGITUDE']] = df.apply(geocode_missing, axis=1)

# Ensure rate limits are respected by pausing between requests
time.sleep(1) 

# Baltimore

In [ ]:
from geopy.distance import geodesic
from folium import FeatureGroup, LayerControl

In [ ]:
# Coordinates for the center of Baltimore City
baltimore_coords = (39.2904, -76.6122)

# Function to calculate distance from Baltimore
def within_15_miles(row):
    if pd.notnull(row['LATITUDE']) and pd.notnull(row['LONGITUDE']):
        hospital_coords = (row['LATITUDE'], row['LONGITUDE'])
        distance = geodesic(baltimore_coords, hospital_coords).miles
        return distance <= 15
    return False

bmore_df = df[df.apply(within_15_miles, axis=1)]
bmore_df = bmore_df.dropna(subset=['LATITUDE', 'LONGITUDE'])

In [ ]:
# Initialize the Map
m = folium.Map(location=[39.2904, -76.6122], zoom_start=9.5)

# Layer 1: Birthing Friendly Designation
birthing_layer = FeatureGroup(name='Birthing Friendly Designation')
for _, row in bmore_df.iterrows():
    color = 'green' if row['Meets criteria for birthing friendly designation'] == 'Y' else 'red'
    
    popup_content = f"""
    {row['Facility Name']}<br>
    Birthing Friendly: {row['Meets criteria for birthing friendly designation']}<br>
    Hospital Rating: {row['Hospital overall rating']}
    """

    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=popup_content,
        icon=folium.Icon(color=color)
    ).add_to(birthing_layer)

birthing_layer.add_to(m)


# Add Custom Legend
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 200px; height: 90px; 
     background-color: white; z-index:9999; font-size:12px;
     border:2px solid grey; border-radius:8px; padding: 10px;">
     <b>Birthing Friendly Hospital</b><br>
     <i class="fa fa-map-marker fa-2x" style="color:green"></i> Yes<br>
     <i class="fa fa-map-marker fa-2x" style="color:red"></i> No
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

radius_meters = 15 * 1609.34

folium.Circle(
    location=baltimore_coords,
    radius=radius_meters,
    color='blue',
    fill=True,
    fill_opacity=0.1,
    popup='15-Mile Radius from Baltimore'
).add_to(m)

# Add a marker for Baltimore center
folium.Marker(
    location=baltimore_coords,
    popup='Baltimore City Center',
    icon=folium.Icon(color='blue')
).add_to(m)

m.save('outputs/baltimore.html')